# InvokeAI in Google Colab

###Introduction

This is a tool to use Google colab to run the AI image generation tool: Invokeai (https://invoke-ai.github.io/InvokeAI/).
This automatically builds itself, It can connect to Google drive to save your images.
It also has the option of running from Google Drive, This takes about 2GB + Models of Google Drive space.

Make sure to enable GPU This should be on by default, but the setting can be found in the menu under: Edit > Notebook Settings > Hardware accelerator > GPU

To start, Click "Runtime" > "Run All". Alternaivly you can click the "play" button on each step below one after the other, No need to wait for the previous steps to finish as they will join a queue.


In [ ]:
#@title 1. Configuration { display-mode: "form" }
#@markdown #Instance Type
#@markdown **Google_Drive** = Stores ALL files in your Google drive, the space it takes up is hevily based on the type and number of models that you are using. but will start up much faster as it does not download InvokeAI at each run. If you have issues with this, please use the "Rebuild" feature in advanced settings, or alternativly delete the whole install directory. <br>
#@markdown **Temporary** (NOT recomended) = Everything is stored in the runtime and is removed when the runtime ends or crashes, make sure to download your images! <br>
Type = "Google_Drive" #@param ['Google_Drive','Temporary'] {type:"string"}
#@markdown <br>
#@markdown **Rough Startup time:** <br>
#@markdown It takes about 5-10 mins to start up, Models are downloaded in the model manager, and can take 2-5 mins each.


#@markdown ---

#@markdown #Connection Type.
#@markdown **NGROK**: (Recomended) Highly stable but needs a little setting up, An NGROK token is required, sign up for free and get one here: https://dashboard.ngrok.com/get-started/your-authtoken Once you have the token, please put it in below.<br>
#@markdown **Localtunnel**: Stable once connected, but sometimes has issues.<br>
#@markdown **Serveo**: less stable, requires no setup, an alternative to Localtunnel.<br>
connection_type = "Localtunnel"  #@param ["Serveo","Localtunnel","NGROK"]
ngrok_token = "None" #@param ['None'] {allow-input: true}


In [ ]:
#@title 2. Models { display-mode: "form" }

#@markdown ###There has been a huge overhaul of model management in InvokeAI V4 and newer.
#@markdown ###All model mangement is now done in-app, the "Model Manager" is found on the left hand side. <br />
#@markdown If you are using Temprory Mode, would you like to Mount Google Drive to import models from?
GDrive_Import = "No" #@param ["Yes","No"]

#@markdown The path to the root of your Google drive will be added as: /content/drive/MyDrive/ You will use that in the "Model manager" to import models.

In [ ]:
#@title 3. Advanced Options { display-mode: "form" }

#@markdown ##Version
#@markdown By default this application uses "The latest official release of InvokeAI" however if you wish to use a specific version please enter its version code below.<br>
#@markdown Please be aware, If you are using a "Google_Drive" version of this program. DOWNGRADING is not officially supported. It may work, it may not, If this does break InvokeAI please DELETE the whole folder /Google_Drive/InvokeAI/invokeai from https://drive.google.com/, If required please backup your "Autoimport" folder. <br>
#@markdown Some older versions can be located in the dropdown. Unless you have a VERY GOOD reason to, I would always use the "Default" mode, but any version 3.0.2a1 or newer should work.
Version = "Default" #@param ["Default","5.0.0","4.2.9","4.0.2","3.2.0","3.1.0","3.0.2.post1","3.0.2a1"] {allow-input: true}

#@markdown ##Rebuild
#@markdown If you are having any issues with a "Google Drive" install of InvokeAI but want to keep your imported models and settings, set this to "Yes" to attempt a repair of the app. <br>
#@markdown Additionally some settings are ignored on subsiquent runs, use this to run those steps again. <br>

Rebuild = "No" #@param ["Yes","No"]

#@markdown ---

#@markdown #Model training.<br />
#@markdown "hollowstrawberry" has an amazing Google colab LoRA maker, it is 100X Better than I could do! It can be found here:<br />
#@markdown Dataset Maker - https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb <br />
#@markdown LoRA Trainer - https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb

In [ ]:
#@title 4. Build and Configure. { display-mode: "form" }
#@markdown Install InvokeAI and it's dependencies, This takes around 6-8 mins.


from IPython.display import clear_output

#Set up temporary storage if running in "Temporary" mode.
if Type == "Temporary":
    file_path = '/content/invokeai'
    noUpdate = '/content/invokeai/noUpdate'
    import os
    if not os.path.exists(file_path):
      os.makedirs(file_path)

#Mount google drive if requested and required.
    if GDrive_Import == "Yes":
      import os
      from google.colab import drive
      if not os.path.exists('/content/drive/'):
        drive.mount('/content/drive')


# Mount and set up Google drive if running in "Google_Drive" mode.
if Type == "Google_Drive":
  file_path = '/content/drive/MyDrive/invokeai/invokeaiapp'
  noUpdate = '/content/drive/MyDrive/invokeai/noUpdate'

  import os
  from google.colab import drive
  if not os.path.exists(file_path):
    drive.mount('/content/drive')
    if not os.path.exists(file_path):
      os.makedirs(file_path)


#Action the rebuild flag.
if Rebuild == "Yes":
  !sudo rm -R {noUpdate}

%cd {file_path}


#Update pip
%cd {file_path}
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python -m pip install --upgrade pip

#Install a dependency
!apt install python3.10-venv



#Create InvokeAI root
import os
os.environ['INVOKEAI_ROOT'] = file_path
if not os.path.exists(file_path):
  os.makedirs(invokeai_root)

#Create the virtual environment + Download default Models
%cd {file_path}

#On 2nd run, Do an "upgrade" to get system variables to load quickly.
if os.path.exists(noUpdate):
  !python -m venv .venv --prompt InvokeAI
  if Version=="Default":
    !source .venv/bin/activate; python -m pip install InvokeAI[xformers] --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu121 --upgrade
  if Version != "Default":
    !source .venv/bin/activate; python -m pip install InvokeAI[xformers]=={Version} --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu121 --upgrade


#PIP First time install of InvokeAI.
if not os.path.exists(noUpdate):
  !python -m venv .venv --prompt InvokeAI
  !source .venv/bin/activate; python -m pip install --upgrade pip

  if Version=="Default":
    !source .venv/bin/activate; python -m pip install InvokeAI[xformers] --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu121
  if Version != "Default":
    !source .venv/bin/activate; python -m pip install InvokeAI[xformers]=={Version} --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu121

  !mkdir {noUpdate}


# Edit invokeai.yaml

#Adjust Ram Cache
#!sed -i 's/ram: 7.5/ram: 10.0/' invokeai.yaml
#!sed -i 's/vram: 0.25/vram: 10.0/' invokeai.yaml

%cd {file_path}

#POST install adjustments
#!pip install requests==2.32.3

#Clear Output
clear_output()

In [ ]:
#@title 5. Start InvokeAI. { display-mode: "form" }
#@markdown ## Starting the App
#@markdown This step takes about 15 seconds to generate your URL but 30 seconds after it is launched before it will work fully! <br>

#@markdown ## Notes about connection types.
#@markdown **NGROK** = Very stable but requires a token, see the "configuration" step for more details.<br>
#@markdown **Localtunnel** = Once it gets going it is quite stable but often has "502" Errors, You must wait for THEM to fix it, please try another connection type.<br>
#@markdown **Serveo** = Almost, always connects however, it can drop your connection if a HTML error occurs, Simply wait for the images to finish generating, stop an re-start this step.



%cd {file_path}
import os

if connection_type == "Serveo":
  !ssh -o StrictHostKeyChecking=no -o ServerAliveInterval=60 -R 80:localhost:9090 serveo.net & . {file_path}/.venv/bin/activate; invokeai-web

if connection_type == "Localtunnel":
  print("How to connect to localtunnel:");
  print("A localtunnel Interface connection is generated here, To use this, please do the following ")
  print("1. Copy this IP address")
  !curl ipv4.icanhazip.com
  print("2. Click the random 'https://XXX-YYY-ZZZ.loca.lt' link that is generated below.")
  print("3. Paste the IP into the provided box and submit. ")
  print(" ")
  print("Note: An error of '502 Bad Gateway' typically is an error at Localtunnels end. A '504 Gateway Time-out' Error means invokeai has not started yet.")
  print(" ")
  !npm install -g localtunnel
  !npx localtunnel --port 9090 & . {file_path}/.venv/bin/activate; invokeai-web

if connection_type == "NGROK":
  if ngrok_token == "None":
    print("You have Selected NGROK but did not supply an NGROK token.")
    print("Falling back to a 'Serveo' connection type.")
    print("Please either add an NGROK token to step 1, re-run step 1, then re-run this step, or just re-run this step to use 'Servio'.")
    connection_type = "Serveo"
  if ngrok_token != "None":
    !pip install pyngrok --quiet
    from pyngrok import ngrok
    ngrok.kill()
    ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(9090).public_url
    print(f'InvokeAI Public URL: {public_url}')
    ! . {file_path}/.venv/bin/activate; invokeai-web